In [1]:
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, InputLayer, LeakyReLU, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
from sklearn.metrics import roc_curve, auc
from dataset_batch import load_train_data, load_test_data
import numpy as np
import tensorflow as tf
np.random.seed(3363)
tf.set_random_seed(3363)

Using TensorFlow backend.


In [2]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

class DenseNetModel():
    
    def __init__(self, input_dim=(224,224,3), output_dim=14, learning_rate=0.00001, epochs=5, drop_out=0.3):

        # parms:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.drop_out = drop_out
    
        # Define DenseNet
        self.model = Sequential()

        base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=self.input_dim)
        
        # Freeze the model first
        base_model.trainable = False

        self.model.add(base_model)

        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(512))
        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(self.output_dim, activation = 'sigmoid'))

        auc_roc = as_keras_metric(tf.metrics.auc)
        recall = as_keras_metric(tf.metrics.recall)
        precision = as_keras_metric(tf.metrics.precision)

        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae', auc_roc, recall, precision])
        self.model.summary()

        # Image augmentation
        self.core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip=False, 
                              height_shift_range=0.05, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range=0.1,
                              fill_mode='nearest',
                              zoom_range=[0.85,1.05])

    # Data standardization
    def standardize(self, x):
        self.core_idg.fit(x)
        return
        
    def fit(self, x, y, vx, vy):

        # fit the data
        print ("Start Training model")
        X_train, y_train, X_test, y_test = x, y, vx, vy
        hist = self.model.fit_generator(
            (self.core_idg.flow(X_train, y_train, batch_size = 25)),
            validation_data = self.core_idg.flow(X_test, y_test), epochs=self.epochs)
        print ("Done Training model")
        return hist
    
    # data did preprocessing            
    def inference(self, x):
        return self.model.predict(x)

    def score(self, x, y):
        
        # make predicition
        y_pred = self.predict(x)
        auc_val = 0.0

        # calculate score
        for idx in range(self.output_dim):
            fpr, tpr, _ = roc_curve(y[:,idx].astype(int), y_pred[:,idx])
            auc_val += auc(fpr, tpr)
        
        return auc_val / self.output_dim

    # pain data without preprocessing
    def predict(self, x):
        x = self.core_idg.standardize(x)
        return self.inference(x)

    def save_weight(self, path = 'baseline.h5'):
        print ("Start Saving model")
        self.model.save(path)
        print ("Done Saving model")
        return

    def load_weight(self, path = 'baseline.h5'):
        print ("Start Loading model")
        self.model.load_weights(path)
        print ("Done Loading model")
        return

In [3]:
if __name__ == "__main__":
    
    model = DenseNetModel(input_dim=(224,224,3), epochs=3)

    X_vali, y_vali = load_train_data(min_cnt=9000, max_cnt=10002)
    for iter in range(30):
        for base in range(0, 9000, 1800):
            X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+1799)
            model.standardize(X_train)
            model.fit(X_train, y_train, X_vali, y_vali)
        print("AVG AUC:", model.score(X_vali, y_vali))

    # Saving model
    model.save_weight()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                7182      
Total params: 7,569,486
Trainable params: 531,982
Non-trainable params: 7,037,504
_________________________________________________________________
Loading Training Images and Labels
0.0 % done
1.0 % done
2.0 % done
3.0 % done
4.0 % done
5.0 % done
6.0 % done
7.0 % done
8.0

/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


11.0 % done
12.0 % done
13.0 % done
14.0 % done
15.0 % done
16.0 % done
17.0 % done
18.0 % done
19.0 % done
20.0 % done
21.0 % done
22.0 % done
23.0 % done
24.0 % done
25.0 % done
26.0 % done
27.0 % done
28.0 % done
29.0 % done
30.0 % done
31.0 % done
32.0 % done
33.0 % done
34.0 % done
35.0 % done
36.0 % done
37.0 % done
38.0 % done
39.0 % done
40.0 % done
41.0 % done
42.0 % done
43.0 % done
44.0 % done
45.0 % done
46.0 % done
47.0 % done
48.0 % done
49.0 % done
50.0 % done
51.0 % done
52.0 % done
53.0 % done
54.0 % done
55.0 % done
56.0 % done
57.0 % done
58.0 % done
59.0 % done
60.0 % done
61.0 % done
62.0 % done
63.0 % done
64.0 % done
65.0 % done
66.0 % done
67.0 % done
68.0 % done
69.0 % done
70.0 % done
71.0 % done
72.0 % done
73.0 % done
74.0 % done
75.0 % done
76.0 % done
77.0 % done
78.0 % done
79.0 % done
80.0 % done
81.0 % done
82.0 % done
83.0 % done
84.0 % done
85.0 % done
86.0 % done
87.0 % done
88.0 % done
89.0 % done
90.0 % done
91.0 % done
92.0 % done
93.0 % done
94.0

Done Training model
Loading Training Images and Labels
0.0 % done
1.0 % done
2.0 % done
3.0 % done
4.0 % done
5.0 % done
6.0 % done
7.0 % done
8.0 % done
9.0 % done
10.0 % done
11.0 % done
12.0 % done
13.0 % done
14.0 % done
15.0 % done
16.0 % done
17.0 % done
18.0 % done
19.0 % done
20.0 % done
21.0 % done
22.0 % done
23.0 % done
24.0 % done
25.0 % done
26.0 % done
27.0 % done
28.0 % done
29.0 % done
30.0 % done
31.0 % done
32.0 % done
33.0 % done
34.0 % done
35.0 % done
36.0 % done
37.0 % done
38.0 % done
39.0 % done
40.0 % done
41.0 % done
42.0 % done
43.0 % done
44.0 % done
45.0 % done
46.0 % done
47.0 % done
48.0 % done
49.0 % done
50.0 % done
51.0 % done
52.0 % done
53.0 % done
54.0 % done
55.0 % done
56.0 % done
57.0 % done
58.0 % done
59.0 % done
60.0 % done
61.0 % done
62.0 % done
63.0 % done
64.0 % done
65.0 % done
66.0 % done
67.0 % done
68.0 % done
69.0 % done
70.0 % done
71.0 % done
72.0 % done
73.0 % done
74.0 % done
75.0 % done
76.0 % done
77.0 % done
78.0 % done
79.0 % 

88.0 % done
89.0 % done
90.0 % done
91.0 % done
92.0 % done
93.0 % done
94.0 % done
95.0 % done
96.0 % done
97.0 % done
98.0 % done
99.0 % done
100.0 % done
Done Loading Training Images and Labels from 1800 to 3599
(1800, 224, 224, 3) (1800, 14)
Start Training model
Epoch 1/3
72/72 [==============================] - 22s 312ms/step - loss: 0.1830 - binary_accuracy: 0.9468 - mean_absolute_error: 0.0900 - auc: 0.7323 - recall: 1.0000 - precision: 0.0498 - val_loss: 0.2284 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0631 - val_auc: 0.7326 - val_recall: 1.0000 - val_precision: 0.0499
Epoch 2/3
72/72 [==============================] - 20s 281ms/step - loss: 0.1736 - binary_accuracy: 0.9475 - mean_absolute_error: 0.0881 - auc: 0.7335 - recall: 1.0000 - precision: 0.0499 - val_loss: 0.2471 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0595 - val_auc: 0.7340 - val_recall: 1.0000 - val_precision: 0.0500
Epoch 3/3
72/72 [==============================] - 20s 283ms/step

...

64.0 % done
65.0 % done
66.0 % done
67.0 % done
68.0 % done
69.0 % done
70.0 % done
71.0 % done
72.0 % done
73.0 % done
74.0 % done
75.0 % done
76.0 % done
77.0 % done
78.0 % done
79.0 % done
80.0 % done
81.0 % done
82.0 % done
83.0 % done
84.0 % done
85.0 % done
86.0 % done
87.0 % done
88.0 % done
89.0 % done
90.0 % done
91.0 % done
92.0 % done
93.0 % done
94.0 % done
95.0 % done
96.0 % done
97.0 % done
98.0 % done
99.0 % done
100.0 % done
Done Loading Training Images and Labels from 7200 to 8999
(1800, 224, 224, 3) (1800, 14)
Start Training model
Epoch 1/3
72/72 [==============================] - 22s 306ms/step - loss: 0.1802 - binary_accuracy: 0.9488 - mean_absolute_error: 0.0872 - auc: 0.7655 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2177 - val_binary_accuracy: 0.9468 - val_mean_absolute_error: 0.0670 - val_auc: 0.7655 - val_recall: 1.0000 - val_precision: 0.0500
Epoch 2/3
72/72 [==============================] - 20s 279ms/step - loss: 0.1673 - binary_accuracy: 0.9500 - me

72/72 [==============================] - 20s 274ms/step - loss: 0.1584 - binary_accuracy: 0.9538 - mean_absolute_error: 0.0786 - auc: 0.7657 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2100 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0726 - val_auc: 0.7657 - val_recall: 1.0000 - val_precision: 0.0500
Epoch 3/3
72/72 [==============================] - 20s 281ms/step - loss: 0.1552 - binary_accuracy: 0.9535 - mean_absolute_error: 0.0773 - auc: 0.7657 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2212 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0775 - val_auc: 0.7657 - val_recall: 1.0000 - val_precision: 0.0500
Done Training model
Loading Training Images and Labels
0.0 % done
1.0 % done
2.0 % done
3.0 % done
4.0 % done
5.0 % done
6.0 % done
7.0 % done
8.0 % done
9.0 % done
10.0 % done
11.0 % done
12.0 % done
13.0 % done
14.0 % done
15.0 % done
16.0 % done
17.0 % done
18.0 % done
19.0 % done
20.0 % done
21.0 % done
22.0 % done
23.0 % done
24.0 % do

31.0 % done
32.0 % done
33.0 % done
34.0 % done
35.0 % done
36.0 % done
37.0 % done
38.0 % done
39.0 % done
40.0 % done
41.0 % done
42.0 % done
43.0 % done
44.0 % done
45.0 % done
46.0 % done
47.0 % done
48.0 % done
49.0 % done
50.0 % done
51.0 % done
52.0 % done
53.0 % done
54.0 % done
55.0 % done
56.0 % done
57.0 % done
58.0 % done
59.0 % done
60.0 % done
61.0 % done
62.0 % done
63.0 % done
64.0 % done
65.0 % done
66.0 % done
67.0 % done
68.0 % done
69.0 % done
70.0 % done
71.0 % done
72.0 % done
73.0 % done
74.0 % done
75.0 % done
76.0 % done
77.0 % done
78.0 % done
79.0 % done
80.0 % done
81.0 % done
82.0 % done
83.0 % done
84.0 % done
85.0 % done
86.0 % done
87.0 % done
88.0 % done
89.0 % done
90.0 % done
91.0 % done
92.0 % done
93.0 % done
94.0 % done
95.0 % done
96.0 % done
97.0 % done
98.0 % done
99.0 % done
100.0 % done
Done Loading Training Images and Labels from 1800 to 3599
(1800, 224, 224, 3) (1800, 14)
Start Training model
Epoch 1/3
72/72 [==============================] 

72/72 [==============================] - 22s 303ms/step - loss: 0.1790 - binary_accuracy: 0.9486 - mean_absolute_error: 0.0860 - auc: 0.7660 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2242 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0639 - val_auc: 0.7660 - val_recall: 1.0000 - val_precision: 0.0500
Epoch 2/3
72/72 [==============================] - 20s 277ms/step - loss: 0.1681 - binary_accuracy: 0.9496 - mean_absolute_error: 0.0852 - auc: 0.7660 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2211 - val_binary_accuracy: 0.9467 - val_mean_absolute_error: 0.0718 - val_auc: 0.7660 - val_recall: 1.0000 - val_precision: 0.0500
Epoch 3/3
72/72 [==============================] - 20s 282ms/step - loss: 0.1668 - binary_accuracy: 0.9492 - mean_absolute_error: 0.0844 - auc: 0.7660 - recall: 1.0000 - precision: 0.0500 - val_loss: 0.2151 - val_binary_accuracy: 0.9465 - val_mean_absolute_error: 0.0763 - val_auc: 0.7660 - val_recall: 1.0000 - val_precision: 0.0500
Done Tra

6.0 % done
7.0 % done
8.0 % done
9.0 % done
10.0 % done
11.0 % done
12.0 % done
13.0 % done
14.0 % done
15.0 % done
16.0 % done
17.0 % done
18.0 % done
19.0 % done
20.0 % done
21.0 % done
22.0 % done
23.0 % done
24.0 % done
25.0 % done
26.0 % done
27.0 % done
28.0 % done
29.0 % done
30.0 % done
31.0 % done
32.0 % done
33.0 % done
34.0 % done
35.0 % done
36.0 % done
37.0 % done
38.0 % done
39.0 % done
40.0 % done
41.0 % done
42.0 % done
43.0 % done
44.0 % done
45.0 % done
46.0 % done
47.0 % done
48.0 % done
49.0 % done
50.0 % done
51.0 % done
52.0 % done
53.0 % done
54.0 % done
55.0 % done
56.0 % done
57.0 % done
58.0 % done
59.0 % done
60.0 % done
61.0 % done
62.0 % done
63.0 % done
64.0 % done
65.0 % done
66.0 % done
67.0 % done
68.0 % done
69.0 % done
70.0 % done
71.0 % done
72.0 % done
73.0 % done
74.0 % done
75.0 % done
76.0 % done
77.0 % done
78.0 % done
79.0 % done
80.0 % done
81.0 % done
82.0 % done
83.0 % done
84.0 % done
85.0 % done
86.0 % done
87.0 % done
88.0 % done
89.0 % d